In [1]:
import numpy as np
import pandas as pd
import pickle
# SQL related packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics, preprocessing
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
# text analysis packages
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

#### Extract claims

In [3]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
claims = pd.DataFrame()

# import the abstract from each table
for year in years:
    # query:
    sql_query = """
    SELECT claims, id, payment_times
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    claims = pd.concat([claims, results], axis = 0)
    
# check size of the data
claims.shape

(12033, 3)

In [4]:
# extract the response variable
# reformat the response variable into binary
y_data = np.zeros(claims.shape[0])
y_data[claims['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.628521565694


#### Spliting train-test data first
Perform tokenization and other preprocessing on the training data alone

In [5]:
x_data = claims['claims'].values

# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size = 0.2, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (9626,)
x_test:  (2407,)
y_train:  (9626,)
y_test:  (2407,)


#### Tokenization and cleaning

In [6]:
def tokenize_cleaning(text):
    # tokenize the text first
    tokens = word_tokenize(text.decode('utf-8'))
    
    # lowercase all the words
    tokens = [w.lower() for w in tokens]
    
    # clean up stop words and punctuations 
    stop_list = stopwords.words('english') + list(string.punctuation)

    tokens_no_stop = [token for token in tokens
                        if token not in stop_list]            
    
#     # extract stem of the words
#     stemmer = PorterStemmer()
#     tokens_stem = [stemmer.stem(token) for token in tokens_no_stop]

    # use lemma instead
    # reason: remove the influence of plural or tense
    # but retain the subtle difference in legal writting
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(token) for token in tokens_no_stop]
    
    # remove numbers (the actual values are not useful)
    tokens_no_num = []
    for token in tokens_lemma:
        try:
            float(token)
        except:
            tokens_no_num.append(token)
    
    return tokens_no_num

In [7]:
# tokenize_clean the training text and testing text separately

cleaned_train = []
for i in range(x_train.shape[0]):
    tokens = tokenize_cleaning(x_train[i])
    cleaned_train.append(' '.join(word for word in tokens))
    
cleaned_test = []
for i in range(x_test.shape[0]):
    tokens = tokenize_cleaning(x_test[i])
    cleaned_test.append(' '.join(word for word in tokens))

In [36]:
# convert to bag-of-words
# min number selected by examining the low-frequency words
vectorizer = CountVectorizer(max_df = 0.4, min_df=2)

# perform a count-based vectorization of the document
word_vec = vectorizer.fit(cleaned_train)
word_counts_train = word_vec.transform(cleaned_train)
word_counts_test = word_vec.transform(cleaned_test)

# convert to array
word_counts_train = word_counts_train.toarray()
word_counts_test = word_counts_test.toarray()

print "Training counts dimension: ", word_counts_train.shape
print "Testing counts dimension: ", word_counts_test.shape

Training counts dimension:  (9626, 13490)
Testing counts dimension:  (2407, 13490)


In [37]:
# still need to remove some number words 
# due to how CountVectorizer treats '-' and '/'

# remove any word with numbers in it
words = word_vec.get_feature_names()
num_word_index = np.zeros(len(words))

for i in range(len(words)):
    word = words[i]
    for j in range(len(word)):
        try:
            float(word[j])
            num_word_index[i] = 1
            break
        except:
            continue
        
print "Number of numerical words detected: ", int(np.sum(num_word_index))

# remove the number words
words_no_num = np.asarray(words)[num_word_index == 0]
word_counts_train = word_counts_train[:, num_word_index == 0]
word_counts_test = word_counts_test[:, num_word_index == 0]

print "Training counts dimension: ", word_counts_train.shape
print "Testing counts dimension: ", word_counts_test.shape

Number of numerical words detected:  554
Training counts dimension:  (9626, 12936)
Testing counts dimension:  (2407, 12936)


In [38]:
# check the total counts for each word
total_counts = np.sum(word_counts_train, axis = 0)

df = pd.DataFrame({
    'word': words_no_num,
    'count': total_counts
})

In [39]:
df.sort('count')

/home/jy/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,count,word
6467,2,lapse
9810,2,refrigerating
882,2,astrocytoma
7798,2,nitroxide
9836,2,regressing
488,2,amending
3925,2,encasing
7794,2,nitrosylated
7788,2,nitroimidazole
3915,2,encapsidating


In [115]:
# tf_vectorizer = TfidfVectorizer()
# tfidf_article = tf_vectorizer.fit_transform(cleaned_text)
# tfidf_article = tfidf_article.toarray()

# print tfidf_article.shape

(12033, 29829)


In [40]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = word_counts_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

word_counts_train = word_counts_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = word_counts_train[y_train == 1, :]
x_not_useful = word_counts_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size, :]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7136, 12936)
(7136,)


In [41]:
# standardize the predictors
scaler = preprocessing.StandardScaler()

x_train_std = scaler.fit_transform(x_train_sub)
x_test_std = scaler.transform(word_counts_test)

In [42]:
# try Naive Bayes with Gaussian Distribution
gnb = GaussianNB()

# fit on the training data
gnb.fit(x_train_sub, y_train_sub)

# predict on the test data
y_pred = gnb.predict(word_counts_test)

# accuracy
print "Testing accuracy: ", np.mean(y_pred == y_test)
print "Confustion matrix:"
metrics.confusion_matrix(y_test, y_pred)

Testing accuracy:  0.505193186539
Confustion matrix:


array([[341, 561],
       [630, 875]])

#### Try TF-IDF

In [51]:
# apply IF-IDF
tf_vectorizer = TfidfVectorizer()
tf_vec = tf_vectorizer.fit(cleaned_train)
tfidf_train = tf_vec.transform(cleaned_train)
tfidf_test = tf_vec.transform(cleaned_test)
# convert to array
tfidf_train = tfidf_train.toarray()
tfidf_test = tfidf_test.toarray()

print tfidf_train.shape

(9626, 28079)


### PCA

In [52]:
# try dimensionality reduction using PCA
pca = PCA()

x_train_pca = pca.fit_transform(tfidf_train)
x_test_pca = pca.transform(tfidf_test)

In [56]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = x_train_pca.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train_pca = x_train_pca[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train_pca[y_train == 1, :]
x_not_useful = x_train_pca[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size, :]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7136, 9626)
(7136,)


In [53]:
# find the cum-variance explained at each level
total_var = np.cumsum(pca.explained_variance_ratio_)
n_pc = np.where((total_var > 0.9) == True)[0][0]

print "The number of PCs that can explain > 90% variability: ", n_pc

The number of PCs that can explain > 90% variability:  3567


In [57]:
### use logistic regression

# call the model function
model = Log()
# parameter tuning
c =  np.logspace(-4, 4, 9)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'accuracy')
# fit on the data
#grid_model = grid_model.fit(x_train_sub, y_train_sub) 
grid_model = grid_model.fit(x_train_sub[:, :n_pc], y_train_sub) 

In [58]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.50322309417
Best parameter:  {'C': 100.0}


In [59]:
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_sub[:, :n_pc], y_train_sub)
y_pred = best_log.predict(x_test_pca[:, :n_pc])

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.497299543
F1 score:  0.565373563218
Precision:  0.615324472244
Recall:  0.52292358804


array([[410, 492],
       [718, 787]])

In [61]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(30, 60, 5)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_sub[:, :n_pc], y_train_sub)
#grid_model = grid_model.fit(x_train_sub, y_train_sub)

In [49]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.502242152466
Best parameter:  {'max_features': 30}


In [50]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_sub[:, :n_pc], y_train_sub)
y_pred = best_rf.predict(x_test_pca[:, :n_pc])

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.502285002077
F1 score:  0.543792840823
Precision:  0.636931311329
Recall:  0.474418604651


array([[495, 407],
       [791, 714]])